In [1]:
import pandas as pd
from urllib import request

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [2]:
needsUpdate = False

In [3]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

class DateProvider:
    
    INTENSIVSTATIONEN_DATE_FORMAT = "%d.%m.%Y, %H:%M Uhr"

    def __init__(self):
        self.lastUpdatedIntensivstationen = None
        self.lastUpdatedOriginal = None

    def needsUpdate(self):
        return self.getLastUpdatedIntensivstationen() < self.getLastUpdatedOriginal()
        
    def getLastUpdatedIntensivstationen(self):
        if self.lastUpdatedIntensivstationen is None:
            htmlContent = requests.get("https://knollfrank.github.io/HowBadIsMyBatch/intensivstationen.html").text
            soup = BeautifulSoup(htmlContent, "lxml")
            dateStr = soup.find(id = "Datenstand").text
            self.lastUpdatedIntensivstationen = datetime.strptime(dateStr, DateProvider.INTENSIVSTATIONEN_DATE_FORMAT)
        
        return self.lastUpdatedIntensivstationen

    def getLastUpdatedOriginal(self):
        if self.lastUpdatedOriginal is None:
            html = self._getOriginalHtml()
            lastUpdatedColumn = 'Letzte Änderung'
            dataFrame = self._asDataFrame(html, lastUpdatedColumn)
            self.lastUpdatedOriginal = dataFrame.loc['Landkreis-Daten', lastUpdatedColumn].to_pydatetime()

        return self.lastUpdatedOriginal

    def _getOriginalHtml(self):
        options = Options()
        options.headless = True
        driver = webdriver.Firefox(options = options)
        driver.get('https://www.intensivregister.de/#/aktuelle-lage/downloads')
        sleep(10)
        innerHTML = driver.execute_script("return document.body.innerHTML")
        driver.quit()
        return innerHTML

    def _asDataFrame(self, html, lastUpdatedColumn):
        dataFrame = pd.read_html(html, parse_dates = [lastUpdatedColumn])[0]
        dataFrame[lastUpdatedColumn] = pd.to_datetime(dataFrame[lastUpdatedColumn], format = "%d.%m.%Y %H:%M Uhr")
        dataFrame.set_index('Name', inplace = True)
        return dataFrame


In [4]:
dateProvider = DateProvider()
print('lastUpdatedIntensivstationen:', dateProvider.getLastUpdatedIntensivstationen())
print('lastUpdatedOriginal:', dateProvider.getLastUpdatedOriginal())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate: ', needsUpdate)

lastUpdatedIntensivstationen: 2022-03-08 12:38:00
lastUpdatedOriginal: 2022-03-09 12:38:00
needsUpdate:  True


In [5]:
from bs4 import BeautifulSoup

def saveLastUpdatedIntensivstationen(lastUpdated):
    file = "../../docs/intensivstationen.html"
    with open(file) as fp:
        soup = BeautifulSoup(fp, 'lxml')

    soup.find(id = "Datenstand").string.replace_with(lastUpdated.strftime(DateProvider.INTENSIVSTATIONEN_DATE_FORMAT))

    with open(file, "w") as fp:
        fp.write(str(soup))

In [6]:
saveLastUpdatedIntensivstationen(dateProvider.getLastUpdatedOriginal())

In [7]:
def readTimeseries(download = False):
    timeSeriesFile = 'zeitreihe-tagesdaten.csv'
    if download:
        _downloadTimeseries(timeSeriesFile)

    timeseries = pd.read_csv(
        timeSeriesFile,
        low_memory = False,
        usecols = ['date', 'bundesland', 'gemeindeschluessel', 'betten_belegt', 'betten_frei'],
        parse_dates = ['date'],
        date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%Y-%m-%d"),
        dtype = {
            'gemeindeschluessel': 'string',
            'bundesland': 'string'
            })
    return timeseries.sort_values(by = 'date', ascending = True)

# download https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv or https://www.intensivregister.de/#/aktuelle-lage/downloads
def _downloadTimeseries(timeSeriesFile):
    request.urlretrieve(
        'https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv',
        timeSeriesFile)


In [8]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

,date,bundesland,gemeindeschluessel,betten_frei,betten_belegt
0,2020-04-24,01,01001,40,38
267,2020-04-24,09,09471,9,9
266,2020-04-24,09,09464,17,23
265,2020-04-24,09,09463,9,25
264,2020-04-24,09,09462,12,51
...,...,...,...,...,...
270815,2022-03-09,06,06437,9,117
270814,2022-03-09,06,06436,2,24
270813,2022-03-09,06,06435,10,74
270811,2022-03-09,06,06433,3,20


In [9]:
def readKreise(download = False):
    kreiseFile = '04-kreise.xlsx'
    if download:
        _downloadKreise(kreiseFile)
    
    kreise = pd.read_excel(
        kreiseFile,
        sheet_name = 'Kreisfreie Städte u. Landkreise',
        header = 5,
        index_col = 0)
    kreise = kreise.rename(columns = {'2': 'Bundesland', 3: 'Kreis', 6: 'Einwohnerzahl'})[['Bundesland', 'Kreis', 'Einwohnerzahl']]
    kreise.index.set_names("Key", inplace = True)
    return kreise

# download https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile or https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html
def _downloadKreise(kreiseFile):
    request.urlretrieve(
        'https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile',
        kreiseFile)


In [10]:
kreise = readKreise(download = False)
kreise

,Bundesland,Kreis,Einwohnerzahl
Key,,,
NaN,NaN,NaN,NaN
01,Schleswig-Holstein,NaN,NaN
01001,Kreisfreie Stadt,"Flensburg, Stadt",89934.0
01002,Kreisfreie Stadt,"Kiel, Landeshauptstadt",246601.0
01003,Kreisfreie Stadt,"Lübeck, Hansestadt",215846.0
...,...,...,...
2) Die Ergebnisse ab Berichtsjahr 2016 sind aufgrund methodischer Änderungen und technischer Weiterentwicklung\n nur bedingt mit den Vorjahreswerten vegleichbar. Erläuterungen dazu finden Sie unter www.destatis.de beim Bevölkerungsstand.,NaN,NaN,NaN
NaN,NaN,NaN,NaN
© Daten (im Auftrag der Herausgebergemeinschaft Statistische Ämter des Bundes und der Länder),NaN,NaN,NaN


In [11]:
class ColumnsAdder:

    def __init__(self, kreise):
        self.kreise = kreise

    def addKreisAndBundeslandAndEinwohnerzahlColumns(self, dataFrame):
        dataFrame_kreise = pd.merge(dataFrame, self.kreise, how = 'left', left_on = 'gemeindeschluessel', right_index = True)
        dataFrame['Kreis'] = dataFrame_kreise['Kreis']
        dataFrame['Einwohnerzahl'] = dataFrame_kreise['Einwohnerzahl']
        return self._addBundeslandColumn(dataFrame)
        
    def _addBundeslandColumn(self, dataFrame):
        return pd.merge(
            dataFrame,
            self._createBundeslandByKeyTable(),
            how = 'left',
            left_on = 'bundesland',
            right_index = True)

    def _createBundeslandByKeyTable(self):
        return self.kreise[self.kreise.index.str.len() == 2][['Bundesland']]


In [12]:
timeSeries = ColumnsAdder(kreise).addKreisAndBundeslandAndEinwohnerzahlColumns(timeSeries)
timeSeries

,date,bundesland,gemeindeschluessel,betten_frei,betten_belegt,Kreis,Einwohnerzahl,Bundesland
0,2020-04-24,01,01001,40,38,"Flensburg, Stadt",89934.0,Schleswig-Holstein
267,2020-04-24,09,09471,9,9,Bamberg,147497.0,Bayern
266,2020-04-24,09,09464,17,23,Hof,45173.0,Bayern
265,2020-04-24,09,09463,9,25,Coburg,40842.0,Bayern
264,2020-04-24,09,09462,12,51,Bayreuth,74048.0,Bayern
...,...,...,...,...,...,...,...,...
270815,2022-03-09,06,06437,9,117,Odenwaldkreis,96754.0,Hessen
270814,2022-03-09,06,06436,2,24,Main-Taunus-Kreis,239264.0,Hessen
270813,2022-03-09,06,06435,10,74,Main-Kinzig-Kreis,421689.0,Hessen
270811,2022-03-09,06,06433,3,20,Groß-Gerau,275807.0,Hessen


In [13]:
kreisValues = timeSeries['Kreis'].drop_duplicates().values
kreisValues

array(['Flensburg, Stadt', 'Bamberg', 'Hof', 'Coburg', 'Bayreuth',
       'Schwandorf', 'Regensburg', 'Neumarkt i.d.OPf.', 'Cham',
       'Amberg-Sulzbach', 'Amberg', 'Dingolfing-Landau',
       'Straubing-Bogen', 'Rottal-Inn', 'Regen', 'Passau', 'Landshut',
       'Kelheim', 'Weiden i.d.OPf.', 'Forchheim', 'Kronach', 'Kitzingen',
       'Haßberge', 'Rhön-Grabfeld', 'Bad Kissingen', 'Aschaffenburg',
       'Würzburg', 'Schweinfurt', 'Weißenburg-Gunzenhausen', 'Roth',
       'Neustadt a.d.Aisch-Bad Windsheim', 'Nürnberger Land',
       'Erlangen-Höchstadt', 'Ansbach', 'Nürnberg', 'Fürth', 'Erlangen',
       'Wunsiedel i.Fichtelgebirge', 'Lichtenfels', 'Kulmbach',
       'Freyung-Grafenau', 'Miltenberg', 'Deggendorf', 'Bodenseekreis',
       'Biberach', 'Alb-Donau-Kreis', 'Ulm, Stadtkreis',
       'Zollernalbkreis', 'Tübingen', 'Reutlingen', 'Waldshut', 'Lörrach',
       'Ravensburg', 'Konstanz', 'Schwarzwald-Baar-Kreis', 'Rottweil',
       'Ortenaukreis', 'Emmendingen', 'Breisgau-Hochsc

In [14]:
def printKreisOptions(kreisValues):
    for kreis in kreisValues:
        printKreisOption(kreis)

def printKreisOption(kreis):
    print('<option value="{kreis}">{kreis}</option>'.format(kreis = kreis))

In [15]:
kreisValues = sorted(kreisValues)
# FK-TODO: die folgenden Optionen in der Datei intensivstationen.html in das select-Element nach "Alle Landkreise" einsetzen 
printKreisOptions(kreisValues)

<option value="Ahrweiler">Ahrweiler</option>
<option value="Aichach-Friedberg">Aichach-Friedberg</option>
<option value="Alb-Donau-Kreis">Alb-Donau-Kreis</option>
<option value="Altenburger Land">Altenburger Land</option>
<option value="Altenkirchen (Westerwald)">Altenkirchen (Westerwald)</option>
<option value="Altmarkkreis Salzwedel">Altmarkkreis Salzwedel</option>
<option value="Altötting">Altötting</option>
<option value="Alzey-Worms">Alzey-Worms</option>
<option value="Amberg">Amberg</option>
<option value="Amberg-Sulzbach">Amberg-Sulzbach</option>
<option value="Ammerland">Ammerland</option>
<option value="Anhalt-Bitterfeld">Anhalt-Bitterfeld</option>
<option value="Ansbach">Ansbach</option>
<option value="Aschaffenburg">Aschaffenburg</option>
<option value="Augsburg">Augsburg</option>
<option value="Aurich">Aurich</option>
<option value="Bad Dürkheim">Bad Dürkheim</option>
<option value="Bad Kissingen">Bad Kissingen</option>
<option value="Bad Kreuznach">Bad Kreuznach</option>
<

In [16]:
import os


class IOUtils:

    def saveDictAsJson(dict, file):
        IOUtils.ensurePath(file)
        with open(file, 'w') as outfile:
            json.dump(dict, outfile)

    @staticmethod
    def ensurePath(file):
        directory = os.path.dirname(file)
        if not os.path.exists(directory):
            os.makedirs(directory)


In [17]:
def getIntensiveCareBeds(timeSeries, kreis = None):
    if kreis is not None:
        return timeSeries[timeSeries['Kreis'] == kreis][['date', 'betten_belegt', 'betten_frei', 'Einwohnerzahl']]
    else:
        return timeSeries.groupby('date').agg(**{
                        'betten_belegt': pd.NamedAgg(column = 'betten_belegt', aggfunc = 'sum'),
                        'betten_frei':   pd.NamedAgg(column = 'betten_frei',   aggfunc = 'sum'),
                        'Einwohnerzahl': pd.NamedAgg(column = 'Einwohnerzahl', aggfunc = 'sum') 
                    }).reset_index()

In [18]:
import json


def getAndPersistIntensiveCareBeds(timeSeries, kreis = None):
    intensiveCareBeds = getIntensiveCareBeds(timeSeries, kreis)
    display(kreis)
    _saveAsJson(intensiveCareBeds, _getFilename(kreis))
    return intensiveCareBeds


def _saveAsJson(intensiveCareBeds, file):
    IOUtils.saveDictAsJson(
        {
            'population': int(intensiveCareBeds.iloc[0]['Einwohnerzahl']),
            'data': _intensiveCareBeds2Dict(intensiveCareBeds),
        },
        file)


def _intensiveCareBeds2Dict(intensiveCareBeds):
    df = intensiveCareBeds[['date', 'betten_belegt', 'betten_frei']]
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    return df.to_dict(orient = "records")


def _getFilename(kreis):
    return '../../docs/data/intensivstationen/intensivstationen-{suffix}.json'.format(suffix = _getSuffix(kreis))


def _getSuffix(kreis):
    return kreis if kreis is not None else 'de'


In [19]:
getAndPersistIntensiveCareBeds(timeSeries)

None

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


,date,betten_belegt,betten_frei,Einwohnerzahl
0,2020-04-24,19237,12270,82401553.0
1,2020-04-25,19100,12290,82401553.0
2,2020-04-26,18617,12694,82401553.0
3,2020-04-27,18803,12537,82360711.0
4,2020-04-28,19345,12207,82504802.0
...,...,...,...,...
680,2022-03-05,20459,4174,82658396.0
681,2022-03-06,20034,4482,82658396.0
682,2022-03-07,20294,4456,82658396.0
683,2022-03-08,20695,4131,82658396.0


In [20]:
for kreis in kreisValues:
    getAndPersistIntensiveCareBeds(timeSeries, kreis)

'Ahrweiler'

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


'Aichach-Friedberg'

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


'Alb-Donau-Kreis'

...

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


'Zweibrücken, kreisfreie Stadt'

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


'Zwickau'

/tmp/ipykernel_42771/2427001320.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')


In [21]:
def publish():
    ! git add -A
    ! git commit -m "updating data"
    ! git push

In [22]:
# publish()